In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import matplotlib
import pandas as pd
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None, cbar=True,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    if cbar:
        cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
        cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    else:
        cbar = None

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_xticklabels(col_labels)
    ax.set_yticks(np.arange(data.shape[0]))
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    #ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "black"),
                     threshold=None,text=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            if text is not None:
                textt = im.axes.text(j, i, text[i, j], **kw)
            else:
                textt = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(textt)

    return texts

In [ ]:
std_thold = 0.05
def return_color(mean, std):
    mean = round(mean, 2)
    std = round(std, 2)
    if mean < 0.85:
        return 3
    elif std > 0.1:
        return 2
    return 0

In [ ]:
from decimal import Decimal
def parse_result(filename, show_tput=True):
    data = pd.read_csv(filename, delimiter= ';', header=None)
    xp100 = np.ones(16)
    xp150 = np.ones(16)

    ratios = [x[15] / x[7] for x in data.values]
    values = [x[7] for x in data.values]

    stds = np.array([x[16] / x[7] for x in data.values])
    std100 = np.ones(16)
    std150 = np.ones(16)
    color100 = np.ones(16)
    color150 = np.ones(16)

    for i in range(16):
        try:
            xp100[i] = ratios[2*i]
            std100[i] = stds[2*i]
            color100[i] = return_color(ratios[2*i], stds[2*i])
        except IndexError:
            xp100[i] = -1.0
            std100[i] = 0.0
            color100[i] = 3
        try:
            xp150[i] = ratios[(2*i)+1]
            std150[i] = stds[(2*i)+1]
            color150[i] = return_color(ratios[(2*i)+1], stds[(2*i)+1])
        except IndexError:
            xp150[i] = -1.0
            std150[i] = 0.0
            color150[i] = 3

    xp100 = xp100.reshape(4, 4)
    xp150 = xp150.reshape(4, 4)
    std100 = std100.reshape(4, 4)
    std150 = std150.reshape(4, 4)
    color100 = color100.reshape(4, 4)
    color150 = color150.reshape(4, 4)

    text100 = np.zeros((4, 4), dtype=object)
    text150 = np.zeros((4, 4), dtype=object)

    for i in range(4):
        for j in range(4):
            s = str(round(values[(j+4*i)*2] / 1000000, 2)) + "\n" if show_tput else ""
            text100[i,j] = s + "x" +  str(round(xp100[i,j], 2)) + "\n(±" + str(round(std100[i,j], 2)) +")"
    for i in range(4):
        for j in range(4):
            s =  str(round(values[(j+4*i)*2+1] / 1000000, 2)) + "\n" if show_tput else ""
            text150[i,j] = s + "x" +  str(round(xp150[i,j], 2)) + "\n(±" + str(round(std150[i,j], 2)) +")"

    return (color100, color150, text100, text150)


In [ ]:
show_tput = True
(q1xp100,  q1xp150,  q1text100,  q1text150)  = parse_result("../verification-results/q1-final.csv", show_tput)
(q2xp100,  q2xp150,  q2text100,  q2text150)  = parse_result("../verification-results/q2-final.csv", show_tput)
(q5xp100,  q5xp150,  q5text100,  q5text150)  = parse_result("../verification-results/q5-final.csv", show_tput)
(q8xp100,  q8xp150,  q8text100,  q8text150)  = parse_result("../verification-results/q8-final.csv", show_tput)
(q11xp100, q11xp150, q11text100, q11text150) = parse_result("../verification-results/q11-final.csv", show_tput)

In [ ]:
memory = ["0.5GB", "1GB", "2GB", "4GB"]
font = "roman" # semibold
fig, ((ax, ax2, ax3, ax4, ax5), (ax6, ax7, ax8, ax9, ax10)) = plt.subplots(2, 5, figsize=(20, 8))
plt.suptitle('Memory per task slot', fontsize=15)
fig.supylabel('Task slots', fontsize=15, x=0.02)
#ax.xaxis.set_label_coords(-0.1, 1.1)

ax6.set_title("Q1", fontsize=15, weight='semibold', y=-0.1)
ax7.set_title("Q2", fontsize=15, weight='semibold', y=-0.1)
ax8.set_title("Q5", fontsize=15, weight='semibold', y=-0.1)
ax9.set_title("Q8", fontsize=15, weight='semibold', y=-0.1)
ax10.set_title("Q11", fontsize=15, weight='semibold', y=-0.1)

colors = ['lightgreen', '#76A089', '#D88FD4', 'lightcoral']
cmap = plt.cm.colors.ListedColormap(colors)

##############################
########### 100% #############
##############################
###########  Q1  #############s
im, cbar = heatmap(q1xp100, [3, 6, 11, 16], memory, ax=ax,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im, text=q1text100, valfmt="{x:.2f}", fontsize=12, fontweight=font)

###########  Q2  #############
im2, cbar2 = heatmap(q2xp100, [3, 4, 5, 6], memory, ax=ax2,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im2, text=q2text100, valfmt="{x:.2f}", fontsize=12, fontweight=font)

###########  Q5  #############
im3, cbar3 = heatmap(q5xp100, [12, 16, 32, 48], memory, ax=ax3,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im3, text=q5text100, valfmt="{x:.2f}", fontsize=12, fontweight=font)

###########  Q8  #############
im4, cbar4 = heatmap(q8xp100, [12, 16, 32, 48], memory, ax=ax4,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im4, text=q8text100, valfmt="{x:.2f}", fontsize=12, fontweight=font)

###########  Q11  ############
im5, cbar4 = heatmap(q11xp100, [12, 16, 32, 48], memory, ax=ax5,
                   cmap=cmap,cbar=False, vmin=0, vmax=4, cbarlabel="tput ratio [% of MST]")
texts = annotate_heatmap(im5, text=q11text100, valfmt="{x:.2f}", fontsize=12, fontweight=font)

##############################
########### 150% #############
##############################

###########  Q1  #############
im6, cbar4 = heatmap(q1xp150, [3, 6, 11, 16], memory, ax=ax6,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im6, text=q1text150, valfmt="{x:.2f}", fontsize=12, fontweight=font)


###########  Q2  #############
im7, cbar4 = heatmap(q2xp150, [3, 4, 5, 6], memory, ax=ax7,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im7, text=q2text150, valfmt="{x:.2f}", fontsize=12, fontweight=font)

###########  Q5  #############
im8, cbar4 = heatmap(q5xp150, [12, 16, 32, 48], memory, ax=ax8,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im8, text=q5text150, valfmt="{x:.2f}", fontsize=12, fontweight=font)

###########  Q8  #############
im9, cbar4 = heatmap(q8xp150, [12, 16, 32, 48], memory, ax=ax9,
                   cmap=cmap,cbar=False, vmin=0, vmax=4)
texts = annotate_heatmap(im9, text=q8text150, valfmt="{x:.2f}", fontsize=12, fontweight=font)

###########  Q11  ############
im10, cbar4 = heatmap(q11xp150, [12, 16, 32, 48], memory, ax=ax10,
                   cmap=cmap,cbar=False, vmin=0, vmax=4, cbarlabel="tput ratio [% of MST]")
texts = annotate_heatmap(im10, text=q11text150, valfmt="{x:.2f}", fontsize=12, fontweight=font)

legend_elements = [Patch(facecolor='lightgreen', edgecolor='w',
                         label='mean > 0.9 & std < 0.1'),
                   Patch(facecolor='orange', edgecolor='w',
                         label='mean > 0.9 & std < 0.1'),
                   Patch(facecolor='mediumpurple', edgecolor='w',
                         label='mean > 0.9 & std < 0.1'),
                  Patch(facecolor='lightcoral', edgecolor='w',
                         label='mean > 0.9 & std < 0.1')]

fig.tight_layout()
plt.savefig("plot-CE-bis.svg", format="svg")
display(fig.get_size_inches())
display(fig.dpi)
plt.show()

